In [37]:
from huggingsound import SpeechRecognitionModel

w2vmodel = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english")


from transformers import pipeline
from transformers import logging
logging.set_verbosity_error() #change'error' to 'warning' or remove this if you want to see the warning
import numpy as np
unmasker = pipeline('fill-mask', model='bert-base-uncased')
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

04/25/2023 18:59:13 - INFO - huggingsound.speech_recognition.model - Loading model...


In [38]:
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    d = [[0] * (n+1) for _ in range(m+1)]
    
    for i in range(m+1):
        d[i][0] = i
    
    for j in range(n+1):
        d[0][j] = j
        
    for j in range(1, n+1):
        for i in range(1, m+1):
            if s[i-1] == t[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = 1 + min(d[i-1][j], d[i][j-1], d[i-1][j-1])
                
    return d[m][n]

In [39]:
def collate(input):
    pun_marks = [",", ".", "?", "!", ";", ":", "-", "—", "(", ")", "[", "]", "{", "}", "'", "\"", "`"]
    output = ""
    Capital = True
    Dash = False
    for i in range(len(input)):
        if input[i] in pun_marks:
            output += input[i]
            if input[i] in [".", "("]:
                Capital = True
            if input[i] in ["-", "'"]:
                Dash = True
            else:
                Dash = False
        else:
            str = ""
            if (Dash == False):
                str += " "
            if Capital:
                str += input[i].capitalize()
                Capital = False
            else:
                str += input[i]
            output += str
    return output

In [48]:
audio_paths = ["karanrecording.mp3"]

transcriptions = w2vmodel.transcribe(audio_paths)

input = transcriptions[0]["transcription"]
input = input.split()
input

100%|██████████| 1/1 [00:04<00:00,  4.15s/it]


['hello',
 'my',
 'name',
 'is',
 'current',
 'and',
 'i',
 'like',
 'computer',
 'scients',
 'the',
 'oh',
 'is',
 'that']

In [45]:
for t in range(1):
    for i in range(len(input)):
        temp = input[i]
        token = tokenizer(temp)['input_ids'][1]
        input[i] = "[MASK]"
        apiint = unmasker(' '.join(input))
        dist = []
        for r in range(5):
            # if (np.abs((apiint[r]['token'] - token)) < 2):
            dist.append(levenshtein_distance(temp, apiint[r]['token_str']))
            # print(dist)
        lindex = 0
        l = dist[0]
        for r in range(5):
            if dist[r] < l:
                lindex = r
                l = dist[r]
        if l <= 2:
            input[i] = apiint[lindex]['token_str']
        else:
            input[i] = temp
            # if (levenshtein_distance(temp, apiint[r]['token_str']) <=2):
            #     input[i] = apiint[r]['token_str']
            #     break
            #     # print(apiint[r]['token_str'])
            # else:
            #     input[i] = temp

for t in range(1):
    inndex = 1
    for i in range(len(input)):
        input.insert(inndex, "[MASK]")
        # print(' '.join(input))
        apiint = unmasker(' '.join(input))
        if (apiint[0]['token'] < 1500):
            input[inndex] = apiint[0]["token_str"]
            inndex += 2
        else:
            del input[inndex]
            inndex += 1

In [47]:
collate(input)
# input.split()

' Hello, my name is current, and, i love computer, scients the, oh, that.'